In [1]:
import geopandas as gpd
import numpy as np

In [2]:
# Read file
city_proper = gpd.read_file('./data/city_proper_over_1000.geojson')
hsr_countries = [
    "China",
    "Spain",
    "France",
    "Germany",
    "Japan",
    "Italy",
    "United Kingdom",
    "Korea, Republic of",
    "Turkey",
    "Finland",
    "Sweden",
    "Uzbekistan",
    "United States",
    "Greece",
    "Russia",
    "Saudi Arabia",
    "Taiwan, China",
    "Austria",
    "Portugal",
    "Poland",
    "Belgium",
    "Morocco",
    "Switzerland",
    "Indonesia",
    "Norway",
    "Netherlands",
    "Serbia",
    "Denmark",
    "Hong Kong, China",
]

In [3]:
city_proper = city_proper[
    [
        "geoname_id",
        "name",
        "ascii_name",
        "country_code",
        "cou_name_en",
        "population",
        "geometry",
    ]
]

city_proper["geoname_id"] = city_proper["geoname_id"].astype("string")
city_proper["name"] = city_proper["name"].astype("string")
city_proper["ascii_name"] = city_proper["ascii_name"].astype("string")
city_proper["country_code"] = city_proper["country_code"].astype("string")
city_proper["cou_name_en"] = city_proper["cou_name_en"].astype("string")
city_proper["population"] = city_proper["population"].astype(int)

In [4]:
city_proper["cou_name_en"] = city_proper["cou_name_en"].fillna("")
city_proper = city_proper[city_proper["cou_name_en"].apply(lambda x: x in hsr_countries)].reset_index(drop=True)
city_proper.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 79439 entries, 0 to 79438
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   geoname_id    79439 non-null  string  
 1   name          79439 non-null  string  
 2   ascii_name    79439 non-null  string  
 3   country_code  79439 non-null  string  
 4   cou_name_en   79439 non-null  string  
 5   population    79439 non-null  int32   
 6   geometry      79439 non-null  geometry
dtypes: geometry(1), int32(1), string(5)
memory usage: 3.9 MB


In [5]:
# Take a subset of only cities with population greater than 50K
city_over_50 = city_proper[city_proper["population"] >= 75000]
city_below_50 = city_proper[(city_proper["population"] < 50000) & (city_proper["population"] > 5000)]

# Want to calculate dist between every city over 50K and every city below 50K
crossed = city_over_50.merge(
    city_below_50,
    how="cross",
)
crossed = gpd.GeoDataFrame(crossed)

In [7]:
crossed.set_geometry("geometry_x")
crossed["geometry_x"] = gpd.GeoSeries(crossed["geometry_x"])
crossed["geometry_y"] = gpd.GeoSeries(crossed["geometry_y"])
crossed["dist"] = crossed["geometry_x"].distance(crossed["geometry_y"])

C:\Users\nicho\AppData\Local\Temp\ipykernel_26992\105447135.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  crossed["dist"] = crossed["geometry_x"].distance(crossed["geometry_y"])


In [12]:
crossed = crossed.set_geometry("geometry_x")

MemoryError: Unable to allocate 493. MiB for an array with shape (1, 64605734) and data type float64

In [ ]:
crossed.crs

AttributeError: The CRS attribute of a GeoDataFrame without an active geometry column is not defined. Use GeoDataFrame.set_geometry to set the active geometry column.

In [1]:
# Select for city pairs closer than 25 miles to main city (over 75K)
crossed[crossed["dist"] > 25]
# If the second (smaller) city is matched to multiple main cities, only keep match with largest main city
crossed.sort_values(by="population_x")
crossed.drop_duplicates(subset="name_y", keep="first")
# Groupby main city, sum population, agg list of names together for record and keep largest city as metro name
crossed.groupby("name_x").agg({"population_y": "sum", "names_y": "list"})
crossed["population"] = crossed["population_x"] + crossed["population_y"]

SyntaxError: invalid syntax (2233595743.py, line 8)